In [59]:
import pandas as pd

In [60]:
phase_lengths = {'early G1': 27.2, 'late G1': 33.7, 'S': 20.5, 'G2': 21.4, 'P/M': 9.8, 'Ana': 10.4, 'T/C': 4.1}
sum(phase_lengths.values())

127.1

In [61]:
mrna_file = '../data/Sup_PivotTable_RNA_forPubli_AllGenes.xlsx'

In [62]:
df = {}

In [70]:
df['SIC1'] = pd.read_excel(mrna_file, sheetname='SIC1', skiprows=1, skipfooter=251, index_col=0, parse_cols='A, C:I')
df['SIC1'].head()

,early G1,late G1,S,G2,P/M,Ana,T/C
# RNA/cell,,,,,,,
0,22,24,22,24,6,1,0
1,35,53,41,45,22,1,0
2,48,74,54,36,20,10,0
3,30,52,22,19,10,6,0
4,10,19,9,10,6,4,0


In [71]:
df['CLN2'] = pd.read_excel(mrna_file, sheetname='CLN2', skiprows=1, skipfooter=260, index_col=0, parse_cols='A, C:I')
df['CLN2'].head()

,early G1,late G1,S,G2,P/M,Ana,T/C
# RNA/cell,,,,,,,
0,123,15,38,105,55,35,11
1,27,8,22,37,12,23,6
2,11,11,9,20,6,3,3
3,12,7,12,4,1,2,1
4,10,2,2,1,0,2,0


In [72]:
df['CLB5'] = pd.read_excel(mrna_file, sheetname='CLB5', skiprows=1, skipfooter=89, index_col=0, parse_cols='A, C:I')
df['CLB5'].head()

,early G1,late G1,S,G2,P/M,Ana,T/C
# RNA/cell,,,,,,,
0,69,35,55,94,43,45,6
1,46,47,47,31,19,26,10
2,27,27,27,12,8,5,1
3,10,25,22,5,0,3,2
4,4,32,12,3,0,4,3
